In [ ]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.io as pio
pio.renderers.default='notebook'
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def make_graph(stock_data, revenue_data, stock):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", FutureWarning)
        fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
        stock_data_specific = stock_data[stock_data.Date <= '2021--06-14']
        revenue_data_specific = revenue_data[revenue_data.Date <= '2021-04-30']
        fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data_specific.Date), y=stock_data_specific.Close.astype("float"), name="Share Price"), row=1, col=1)
        fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data_specific.Date), y=revenue_data_specific.Revenue.astype("float"), name="Revenue"), row=2, col=1)
        fig.update_xaxes(title_text="Date", row=1, col=1)
        fig.update_xaxes(title_text="Date", row=2, col=1)
        fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
        fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
        fig.update_layout(showlegend=False,
        height=900,
        title=stock,
        xaxis_rangeslider_visible=True)
        fig.show()

In [ ]:
tesla = yf.Ticker('TSLA')
tesla_data = pd.DataFrame(tesla.history(period='max'))
tesla_data.reset_index(inplace=True)
tesla_data.head()

In [ ]:
html_data = requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/revenue.htm')
html_data = html_data.text

In [ ]:
tesla_revenue = pd.DataFrame(columns=['Date', 'Revenue'])
soup = BeautifulSoup(html_data, 'html5lib')
for row in soup.find('tbody').find_all('tr'):
    col = row.find_all('td')
    date = col[0].text
    revenue = col[1].text

    tempdf = pd.DataFrame({'Date': [date], 'Revenue': [revenue]})
    tesla_revenue = pd.DataFrame(pd.concat([tesla_revenue, tempdf], ignore_index=True))

tesla_revenue.tail()

In [ ]:
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace(',', "")
tesla_revenue["Revenue"] = tesla_revenue['Revenue'].str.replace('$', "")

print(tesla_revenue)
tesla_revenue.dropna(inplace=True)
tesla_revenue = tesla_revenue[tesla_revenue['Revenue'] != ""]

In [ ]:
tesla_revenue.tail()

In [ ]:
gme = yf.Ticker('GME')
gme_data = pd.DataFrame(gme.history(period='max'))
gme_data.reset_index(inplace=True)
gme_data.head()

In [ ]:
html_data = requests.get('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-PY0220EN-SkillsNetwork/labs/project/stock.html')
soup = BeautifulSoup(html_data.text, 'html5lib')

In [ ]:
gme_revenue = pd.DataFrame(columns=['Date', 'Revenue'])
for row in soup.find('tbody').find_all('tr'):
    col = row.find_all('td')
    date = col[0].text
    revenue = col[1].text
    tempdf = pd.DataFrame({'Date': [date], 'Revenue': [revenue]})
    gme_revenue = pd.DataFrame(pd.concat([gme_revenue, tempdf], ignore_index=True))

gme_revenue.tail()

In [ ]:
import warnings
make_graph(tesla_data, tesla_revenue, 'Tesla')